## Import necessary modules and data

In [1]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from keras_metrics import precision
from sklearn.model_selection import cross_validate
from livelossplot import PlotLossesKeras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import Dropout
from keras_metrics import precision, binary_recall, f1_score
import random



Using TensorFlow backend.


## Wczytujemy poprzednio utworzone zbiory

In [2]:
X = pd.read_csv('../data/cleaned_data.csv', index_col=0)
y = pd.read_csv('../data/labels.csv', header = None)

X_test = pd.read_csv('../data/X_test.csv', index_col=0)
X_train = pd.read_csv('../data/X_train.csv', index_col=0)
X_train_std = pd.read_csv('../data/X_train_std.csv', index_col=0)
X_test_std = pd.read_csv('../data/X_test_std.csv', index_col=0)
y_train = pd.read_csv('../data/y_train.csv', index_col=0)
y_test = pd.read_csv('../data/y_test.csv', index_col=0)

Zbuduję przykładową sieć, która składa się z warstwy wejściowej, jednej warstwy ukrytej i  warstwy wyjściowej.
Kompilujemy model przy użyciu 

loss_function='binary_crossentropy';

optimizer='Adam'.


Naszą główną metryką będzie **precyzja** oznaczająca procent osób prawidłowie określonych jako zagrożone odejściem.

## Pierwsze podejście do modelu

In [3]:
model = Sequential()


model.add(Dense(11, activation='relu', input_shape=(10,)))
 
model.add(Dense(121, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[precision(),'accuracy', binary_recall(), f1_score()])
                   
model.fit(X_train_std, y_train,epochs=3, batch_size=1, verbose=1, validation_data=(X_test,y_test))

model.evaluate(X_test_std,y_test)

tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn
Train on 6700 samples, validate on 3300 samples
Epoch 1/3
6700/6700 [==============================] - 8s 1ms/step - loss: 0.4192 - precision: 0.5743 - accuracy: 0.8255 - recall: 0.1203 - f1_score: 0.1922 - val_loss: 7615.6952 - val_precision: 0.0000e+00 - val_accuracy: 0.7964 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 2/3
6700/6700 [==============================] - 8s 1ms/step - loss: 0.3623 - precision: 0.7042 - accuracy: 0.85

[0.3599678965951457,
 0.6786537170410156,
 0.8427272439002991,
 0.46117037534713745,
 0.5488515496253967]

## Sprawdźmy skuteczność naszej predykcji raportem klasyfikacyjnym


In [4]:
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test,y_pred)
report1 = classification_report(y_test,y_pred,output_dict=True)
print(report)
print(report1['0']['precision'])


              precision    recall  f1-score   support

           0       0.87      0.94      0.91      2628
           1       0.67      0.46      0.54       672

   micro avg       0.84      0.84      0.84      3300
   macro avg       0.77      0.70      0.72      3300
weighted avg       0.83      0.84      0.83      3300

0.8711721224920802


### Sprawdźmy, jak prezentują się wyniki w zależności od liczby neuronów w warstwie ukrytej.

In [5]:
nodes = [10, 30, 50, 70, 90, 110]
wyniki = {}
for i in nodes:
    model = 0

    model = Sequential()

    model.add(Dense(11, activation='relu', input_shape=(10,)))

    model.add(Dense(i, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[precision(), 'accuracy'])

    model.fit(X_train_std, y_train, epochs=5, batch_size=1, verbose=1, validation_data=(X_test, y_test))
    print("Zakończono trenowanie modelu z {0} węzłami.".format(i))
    model.evaluate(X_test_std, y_test)
    y_pred = model.predict_classes(X_test_std)
    report = classification_report(y_test, y_pred, output_dict=True)
    wyniki.update({i: [report['0']['precision'], report['1']['precision']]})

print(wyniki)


tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
Train on 6700 samples, validate on 3300 samples
Epoch 1/5
6700/6700 [==============================] - 7s 1ms/step - loss: 0.4352 - precision: 0.4097 - accuracy: 0.8125 - val_loss: 2036.4791 - val_precision: 0.0000e+00 - val_accuracy: 0.7964
Epoch 2/5
6700/6700 [==============================] - 7s 1ms/step - loss: 0.3700 - precision: 0.7522 - accuracy: 0.8521 - val_loss: 966.0483 - val_precision: 0.3091 - val_accuracy: 0.7873
Epoch 3/5
6700/6700 [==============================] - 7s 1ms/step - loss: 0.3592 - precision: 0.7195 - accuracy: 0.8512 - val_loss: 1089.2046 - val_precision: 0.0000e+00 - val_accuracy: 0.7964
Epoch 4/5
6700/6700 [==============================] - 7s 1ms/step - loss: 0.3511 - precision: 0.7598 - accuracy: 0.8581 - val_loss: 1254.0022 - val_precision: 0.0000e+00 - val_accuracy: 0.7964
Epoch 5/5
6700/6700 [======================

## Wybieramy model z 90 węzłami - dał najlepsze wyniki

In [13]:
model = Sequential(name='model_1')

model.add(Dense(10, activation='relu', input_shape=(10,)))
model.add(Dropout(0.1, ))

model.add(Dense(90, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[precision(),'accuracy'])

early_stopping_monitor = EarlyStopping(patience=2)
model.fit(X_train_std, y_train, epochs=30, batch_size=1, verbose=1, validation_data=(X_test, y_test),
          callbacks=[early_stopping_monitor])

model.evaluate(X_test_std, y_test)
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test, y_pred)
report1 = classification_report(y_test, y_pred, output_dict=True)
print(report)



tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
Train on 6700 samples, validate on 3300 samples
Epoch 1/30
6700/6700 [==============================] - 9s 1ms/step - loss: 0.4417 - precision: 0.4486 - accuracy: 0.8154 - val_loss: 7561.1996 - val_precision: 0.0000e+00 - val_accuracy: 0.7964
Epoch 2/30
6700/6700 [==============================] - 9s 1ms/step - loss: 0.3826 - precision: 0.7359 - accuracy: 0.8484 - val_loss: 5587.9512 - val_precision: 0.0000e+00 - val_accuracy: 0.7964
Epoch 3/30
6700/6700 [==============================] - 8s 1ms/step - loss: 0.3708 - precision: 0.7504 - accuracy: 0.8509 - val_loss: 2732.0952 - val_precision: 0.0000e+00 - val_accuracy: 0.7964
Epoch 4/30
6700/6700 [==============================] - 9s 1ms/step - loss: 0.3663 - precision: 0.7312 - accuracy: 0.8510 - val_loss: 3017.3058 - val_precision: 0.0000e+00 - val_accuracy: 0.7964
Epoch 5/30
6700/6700 [============

## Zapisujemy wytrenowany model

In [14]:
model.save("../saved_models/model_1.h5")

### autor: Tomasz Sołtysiak